# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from googletrans import Translator
pd.set_option('display.max_columns', 1000000, 'display.max_rows', 1000000)

# File descriptions
* **sales_train.csv** - the training set. Daily historical data from January 2013 to October 2015.
* **test.csv** - the test set. You need to forecast the sales for these shops and products for November 2015.
* **sample_submission.csv** - a sample submission file in the correct format.
* **items.csv** - supplemental information about the items/products.
* **item_categories.csv**  - supplemental information about the items categories.
* **shops.csv**- supplemental information about the shops.

# Data fields
* **ID** - an Id that represents a (Shop, Item) tuple within the test set
* **shop_id** - unique identifier of a shop
* **item_id** - unique identifier of a product
* **item_category_id** - unique identifier of item category
* **item_cnt_day** - number of products sold. You are predicting a monthly amount of this measure
* **item_price** - current price of an item
* **date** - date in format dd/mm/yyyy
* **date_block_num** - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33
* **item_name** - name of item
* **shop_name** - name of shop
* **item_category_name** - name of item category

# Reading Data Set

In [78]:
sales_train = pd.read_csv("sales_train.csv")
items = pd.read_csv("items.csv")
item_categories = pd.read_csv("item_categories.csv") 
shops = pd.read_csv("shops.csv")

# Getting basic insight of data frames

In [79]:
print(" sales_train -> ", sales_train.shape, "\n",
     "items -> ", items.shape, "\n",
     "item_categories -> ", item_categories.shape, "\n",
     "shops -> ", shops.shape)

 sales_train ->  (2935849, 6) 
 items ->  (22170, 3) 
 item_categories ->  (84, 2) 
 shops ->  (60, 2)


### ----------------------------------------->>     sales_train  <<------------------------------------------------------###

In [80]:
sales_train.columns, len(sales_train.columns)

(Index(['date', 'date_block_num', 'shop_id', 'item_id', 'item_price',
        'item_cnt_day'],
       dtype='object'), 6)

In [81]:
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [82]:
sales_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
date              object
date_block_num    int64
shop_id           int64
item_id           int64
item_price        float64
item_cnt_day      float64
dtypes: float64(2), int64(3), object(1)
memory usage: 134.4+ MB


In [83]:
sales_train.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06
mean,1.456991e+01,3.300173e+01,1.019723e+04,8.908532e+02,1.242641e+00
std,9.422988e+00,1.622697e+01,6.324297e+03,1.729800e+03,2.618834e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01
25%,7.000000e+00,2.200000e+01,4.476000e+03,2.490000e+02,1.000000e+00
50%,1.400000e+01,3.100000e+01,9.343000e+03,3.990000e+02,1.000000e+00
75%,2.300000e+01,4.700000e+01,1.568400e+04,9.990000e+02,1.000000e+00
max,3.300000e+01,5.900000e+01,2.216900e+04,3.079800e+05,2.169000e+03


### ----------------------------------------->>     items    <<------------------------------------------------------###

In [84]:
items.columns, len(items.columns)

(Index(['item_name', 'item_id', 'item_category_id'], dtype='object'), 3)

In [85]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [86]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22170 entries, 0 to 22169
Data columns (total 3 columns):
item_name           22170 non-null object
item_id             22170 non-null int64
item_category_id    22170 non-null int64
dtypes: int64(2), object(1)
memory usage: 519.7+ KB


In [87]:
items.describe()

,item_id,item_category_id
count,22170.00000,22170.000000
mean,11084.50000,46.290753
std,6400.07207,15.941486
min,0.00000,0.000000
25%,5542.25000,37.000000
50%,11084.50000,40.000000
75%,16626.75000,58.000000
max,22169.00000,83.000000


### ----------------------------------------->>     item_categories    <<------------------------------------------------------###

In [88]:
item_categories.columns, len(item_categories.columns)

(Index(['item_category_name', 'item_category_id'], dtype='object'), 2)

In [89]:
item_categories.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [90]:
item_categories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 2 columns):
item_category_name    84 non-null object
item_category_id      84 non-null int64
dtypes: int64(1), object(1)
memory usage: 1.4+ KB


In [91]:
item_categories.describe()

,item_category_id
count,84.000000
mean,41.500000
std,24.392622
min,0.000000
25%,20.750000
50%,41.500000
75%,62.250000
max,83.000000


### ----------------------------------------->>     shops    <<------------------------------------------------------###

In [92]:
shops.columns, len(shops.columns)

(Index(['shop_name', 'shop_id'], dtype='object'), 2)

In [93]:
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [94]:
shops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 2 columns):
shop_name    60 non-null object
shop_id      60 non-null int64
dtypes: int64(1), object(1)
memory usage: 1.0+ KB


In [95]:
shops.describe()

,shop_id
count,60.000000
mean,29.500000
std,17.464249
min,0.000000
25%,14.750000
50%,29.500000
75%,44.250000
max,59.000000


### Concatinating dataframes

In [96]:
sales_train_CC_items = sales_train.merge(items, 
                                         on="item_id")
sales_train_CC_items_CC_item_categories = sales_train_CC_items.merge(item_categories, 
                                                                     on="item_category_id")
sales_train_CC_items_CC_item_categories_CC_shops = sales_train_CC_items_CC_item_categories.merge(shops, 
                                                                                                 on="shop_id")
print(sales_train_CC_items_CC_item_categories_CC_shops.shape)
print(sales_train_CC_items_CC_item_categories_CC_shops.columns, 
      len(sales_train_CC_items_CC_item_categories_CC_shops.columns))

(2935849, 10)
Index(['date', 'date_block_num', 'shop_id', 'item_id', 'item_price',
       'item_cnt_day', 'item_name', 'item_category_id', 'item_category_name',
       'shop_name'],
      dtype='object') 10


In [97]:
sales_train_CC_items_CC_item_categories_CC_shops.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2935849 entries, 0 to 2935848
Data columns (total 10 columns):
date                  object
date_block_num        int64
shop_id               int64
item_id               int64
item_price            float64
item_cnt_day          float64
item_name             object
item_category_id      int64
item_category_name    object
shop_name             object
dtypes: float64(2), int64(4), object(4)
memory usage: 246.4+ MB


In [98]:
sales_train_CC_items_CC_item_categories_CC_shops.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id
count,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06
mean,1.456991e+01,3.300173e+01,1.019723e+04,8.908532e+02,1.242641e+00,4.000138e+01
std,9.422988e+00,1.622697e+01,6.324297e+03,1.729800e+03,2.618834e+00,1.710076e+01
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01,0.000000e+00
25%,7.000000e+00,2.200000e+01,4.476000e+03,2.490000e+02,1.000000e+00,2.800000e+01
50%,1.400000e+01,3.100000e+01,9.343000e+03,3.990000e+02,1.000000e+00,4.000000e+01
75%,2.300000e+01,4.700000e+01,1.568400e+04,9.990000e+02,1.000000e+00,5.500000e+01
max,3.300000e+01,5.900000e+01,2.216900e+04,3.079800e+05,2.169000e+03,8.300000e+01


### Creating copy of final merged data set

In [99]:
finalDF = sales_train_CC_items_CC_item_categories_CC_shops.copy()
finalDF.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,item_category_name,shop_name
0,02.01.2013,0,59,22154,999.0,1.0,ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир"""
1,26.04.2013,3,59,944,150.0,1.0,2012 (BD),37,Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир"""
2,26.06.2013,5,59,944,199.5,1.0,2012 (BD),37,Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир"""
3,20.07.2013,6,59,944,199.5,1.0,2012 (BD),37,Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир"""
4,14.09.2013,8,59,944,299.0,2.0,2012 (BD),37,Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир"""


In [100]:
finalDF.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id
count,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06
mean,1.456991e+01,3.300173e+01,1.019723e+04,8.908532e+02,1.242641e+00,4.000138e+01
std,9.422988e+00,1.622697e+01,6.324297e+03,1.729800e+03,2.618834e+00,1.710076e+01
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01,0.000000e+00
25%,7.000000e+00,2.200000e+01,4.476000e+03,2.490000e+02,1.000000e+00,2.800000e+01
50%,1.400000e+01,3.100000e+01,9.343000e+03,3.990000e+02,1.000000e+00,4.000000e+01
75%,2.300000e+01,4.700000e+01,1.568400e+04,9.990000e+02,1.000000e+00,5.500000e+01
max,3.300000e+01,5.900000e+01,2.216900e+04,3.079800e+05,2.169000e+03,8.300000e+01


### Removing missing values i.e -1

In [101]:
finalDF[(finalDF.date_block_num == -1)]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,item_category_name,shop_name


In [102]:
finalDF = finalDF[~(finalDF.item_cnt_day == -1)]
finalDF = finalDF[~(finalDF.item_price == -1)]

# Feature Extraction

Splitting date based on dots

In [103]:
dateSplit = finalDF.date.str.split(".", expand=True)
dateSplit.shape

(2928596, 3)

In [104]:
dateSplit.columns = ["day", "month", "year"]
dateSplit.head()

,day,month,year
0,02,01,2013
1,26,04,2013
2,26,06,2013
3,20,07,2013
4,14,09,2013


In [105]:
finalDF = pd.concat([finalDF, dateSplit], axis = 1)#.drop('date', axis=1)

In [106]:
finalDF.columns

Index(['date', 'date_block_num', 'shop_id', 'item_id', 'item_price',
       'item_cnt_day', 'item_name', 'item_category_id', 'item_category_name',
       'shop_name', 'day', 'month', 'year'],
      dtype='object')

### translating Russian text to English text

Converting **item_category_name**

In [107]:
unique_item_category_name = pd.unique(finalDF.item_category_name)
unique_item_category_name = list(unique_item_category_name)
unique_item_category_name

['Кино - Blu-Ray',
 'Музыка - Винил',
 'Музыка - CD фирменного производства',
 'Музыка - Музыкальное видео',
 'Музыка - CD локального производства',
 'Игры - XBOX 360',
 'Игры - PS3',
 'Игры PC - Дополнительные издания',
 'Игры PC - Стандартные издания',
 'Игры - PSP',
 'Кино - DVD',
 'Программы - Для дома и офиса',
 'Книги - Методические материалы 1С',
 'Игры PC - Коллекционные издания',
 'Игры - PSVita',
 'Подарки - Развитие',
 'Программы - 1С:Предприятие 8',
 'Программы - Обучающие',
 'Музыка - MP3',
 'Музыка - Подарочные издания',
 'Аксессуары - PSP',
 'Подарки - Гаджеты, роботы, спорт',
 'Книги - Аудиокниги',
 'Игровые консоли - XBOX 360',
 'Аксессуары - PS3',
 'Аксессуары - PS4',
 'Аксессуары - PSVita',
 'Карты оплаты - PSN',
 'Карты оплаты - Live!',
 'Аксессуары - XBOX 360',
 'Кино - Blu-Ray 3D',
 'Игры - Аксессуары для игр',
 'Игровые консоли - PSVita',
 'Книги - Аудиокниги 1С',
 'Кино - Коллекционное',
 'Подарки - Открытки, наклейки',
 'Игровые консоли - PS3',
 'Подарки - Суве

In [108]:
[re.split('[,]', i) for i in unique_item_category_name]

[['Кино - Blu-Ray'],
 ['Музыка - Винил'],
 ['Музыка - CD фирменного производства'],
 ['Музыка - Музыкальное видео'],
 ['Музыка - CD локального производства'],
 ['Игры - XBOX 360'],
 ['Игры - PS3'],
 ['Игры PC - Дополнительные издания'],
 ['Игры PC - Стандартные издания'],
 ['Игры - PSP'],
 ['Кино - DVD'],
 ['Программы - Для дома и офиса'],
 ['Книги - Методические материалы 1С'],
 ['Игры PC - Коллекционные издания'],
 ['Игры - PSVita'],
 ['Подарки - Развитие'],
 ['Программы - 1С:Предприятие 8'],
 ['Программы - Обучающие'],
 ['Музыка - MP3'],
 ['Музыка - Подарочные издания'],
 ['Аксессуары - PSP'],
 ['Подарки - Гаджеты', ' роботы', ' спорт'],
 ['Книги - Аудиокниги'],
 ['Игровые консоли - XBOX 360'],
 ['Аксессуары - PS3'],
 ['Аксессуары - PS4'],
 ['Аксессуары - PSVita'],
 ['Карты оплаты - PSN'],
 ['Карты оплаты - Live!'],
 ['Аксессуары - XBOX 360'],
 ['Кино - Blu-Ray 3D'],
 ['Игры - Аксессуары для игр'],
 ['Игровые консоли - PSVita'],
 ['Книги - Аудиокниги 1С'],
 ['Кино - Коллекционное'],

In [ ]:
translator = Translator()
translatedList = translator.translate(unique_item_category_name, dest='en')

In [ ]:
translatedEnglishList = []
for i in translatedList:
    translatedEnglishList.append((i.origin, i.text))
translatedEnglishListdf = pd.DataFrame({"item_category_name": [translatedEnglishList[k][0] for k in range(len(translatedEnglishList))],
                                       "en_item_category_name": [translatedEnglishList[k][1] for k in range(len(translatedEnglishList))]
                                       })

In [ ]:
translatedEnglishListdf[translatedEnglishListdf['item_category_name'] == translatedEnglishListdf['en_item_category_name']]

In [ ]:
translatedEnglishListdf.iloc[73]['en_item_category_name'] = "Books - Travel Guides"

In [ ]:
finalDF = finalDF.merge(translatedEnglishListdf, on="item_category_name")

In [ ]:
finalDF.drop('item_category_name', axis=1, inplace=True)

Converting **shop_name**

In [ ]:
unique_shop_name = pd.unique(finalDF.shop_name)
unique_shop_name = list(unique_shop_name)
unique_shop_name

In [ ]:
translatedList_shop_name = translator.translate(unique_shop_name, dest='en')

In [ ]:
translatedEnglishList_shop_name = []
for i in translatedList_shop_name:
    translatedEnglishList_shop_name.append((i.origin, i.text))
translatedEnglishList_shop_namedf = pd.DataFrame({"shop_name": [translatedEnglishList_shop_name[k][0] for k in range(len(translatedEnglishList_shop_name))],
                                       "en_shop_name": [translatedEnglishList_shop_name[k][1] for k in range(len(translatedEnglishList_shop_name))]
                                       })

In [72]:
translatedEnglishList_shop_namedf[translatedEnglishList_shop_namedf['shop_name'] == \
                                  translatedEnglishList_shop_namedf['en_shop_name']]

,shop_name,en_shop_name


In [73]:
finalDF = finalDF.merge(translatedEnglishList_shop_namedf, on="shop_name")
finalDF.drop('shop_name', axis=1, inplace=True)

KeyError: 'shop_name'

In [74]:
finalDF.loc[finalDF.item_name.str.contains('xbox', case = False), ['item_category_id', 'item_name']].head(100)

ValueError: Location based indexing can only have [labels (MUST BE IN THE INDEX), slices of labels (BOTH endpoints included! Can be slices of integers if the index is integers), listlike of labels, boolean] types

Converting **item_name**

In [ ]:
unique_item_name = pd.unique(finalDF.item_name)
unique_item_name = list(unique_item_name)
unique_item_name

In [ ]:
len(unique_item_name)

In [ ]:
unique_item_name_df = pd.DataFrame({'item_name': unique_item_name})

In [ ]:
finalDF.head()

In [ ]:
unique_item_name_df['item_category'] = 0
item_type_check = {'pc': 1,
                   'PC': 1,
                   ' MAC': 2,
                   '[[(]MAC': 2,
                   'Mac': 2,
                   'xbox': 3,
                   'XBOX': 3,
                   'X-Box': 3,
                   'X360': 3,
                   'ps[0-9]': 4,
                   'Ps[0-9]': 4,
                   'PS[0-9]': 4,
                   'Android': 5,
                   'android': 5,
                   'ANDROID': 5,
                   'PSP': 6,
                   'psp': 6,
                   'Psp': 6,
                   'BD': 7,
                   'КНИГА': 8,
                   'mp3': 9,
                   'N1NTENDО': 10,
                   'CD\+\DVD': 11,
                   'windows': 12,
                   'Windows': 12,
                   'WINDOWS': 12,
                   'linux': 13,
                   'Linux': 13,
                   'LINUX': 13
                   
}

In [ ]:
item_type_check_correction = {'Machine': 0,
                              '[0-9]CD': 11
                             }

In [ ]:
for platform, category in item_type_check.items():
    unique_item_name_df.item_category[unique_item_name_df.item_name.str.contains(platform, 
                                                                                 regex=True)] = category

In [ ]:
for platform, category in item_type_check_correction.items():
    unique_item_name_df.item_category[unique_item_name_df.item_name.str.contains(platform,
                                                                                 case = False,
                                                                                 regex=True)] = category

In [ ]:
unique_item_name_df[unique_item_name_df.item_name.str.contains('mac', case=False, regex=True)].head(1000)

In [ ]:
unique_item_name_df[unique_item_name_df.item_category == 0].shape

In [ ]:
unique_item_name_df[unique_item_name_df.item_category == 0].head(1000)

In [ ]:
unique_item_name_df.iloc[2909:2911]

Converting item name from russian to english

In [ ]:
# test = []
# global k
# k = -1
# def iterOver_unique_item_name(k):
#     try:
#         for i in range(k, len(unique_item_name)):
#             k = k + 1
#             print(k)
#             test.append(translator.translate(unique_item_name[i]).text)
            
#     except:
#         test.append(unique_item_name[k-1])
#         k = k + 1
#         print(k)
#         iterOver_unique_item_name(k)
#         pass
        
# iterOver_unique_item_name(k)

In [ ]:
# translatedEnglishList_item_name = []
# for i in translatedList_item_name:
#     translatedEnglishList_item_name.append((i.origin, i.text))
# translatedEnglishList_item_namedf = pd.DataFrame({"shop_name": [translatedEnglishList_item_name[k][0] for k in range(len(translatedEnglishList_item_name))],
#                                        "en_shop_name": [translatedEnglishList_item_name[k][1] for k in range(len(translatedEnglishList_item_name))]
#                                        })

In [65]:
# translatedEnglishList_item_namedf[translatedEnglishList_item_namedf['shop_name'] == \
#                                   translatedEnglishList_item_namedf['en_shop_name']]

Dropping columns

In [66]:
columnslist = list(finalDF.columns)

In [67]:
for column in columnslist:
    print(column, " -- > \n", pd.unique(finalDF[column]), "\n", len(pd.unique(finalDF[column])), "\n \n")

date_block_num  -- > 
 [ 0  3  5  6  8  9 10 12 14 15 16 18 20 26  1 30 22 23  7 17 25  4 11 24
 27  2 19 32 13 21 28 29 31 33] 
 34 
 

shop_id  -- > 
 [59 24 27 25  6 15  7 31 54 42 38 37 46 44 51 16  1  0 28 57 23 58 56 35
 47 21 22 19 18  8 30 41 45 32 29 50 53 43 17  2 52 13 14  3 10 12 26  5
  4 39 48 49 34 11 36 33  9 40 20 55] 
 60 
 

item_id  -- > 
 [22154   944   965 ... 13091 13094 13092] 
 21804 
 

item_price  -- > 
 [ 999.          150.          199.5        ... 2266.33333333 2485.28571429
  912.75      ] 
 19977 
 

item_cnt_day  -- > 
 [ 1.000e+00  2.000e+00  3.000e+00  4.000e+00  5.000e+00  2.000e+01
  1.400e+01  6.000e+00  1.000e+01  9.000e+00  8.000e+00  7.000e+00
  2.600e+01  3.900e+01  2.800e+01  1.300e+01  1.700e+01  2.700e+01
  1.600e+01  1.900e+01  1.200e+01  3.100e+01  1.100e+01  1.500e+01
  8.000e+01  1.800e+01  2.100e+01  5.600e+01 -2.000e+00  3.400e+01
  2.200e+01  2.900e+01  4.500e+01  3.200e+01  8.300e+01  3.800e+01
  2.300e+01  2.400e+01  2.500e+01  5.20

### Dropping columns

In [68]:
finalDF.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,day,month,year,en_item_category_name,en_shop_name
0,0,59,22154,999.0,1.0,ЯВЛЕНИЕ 2012 (BD),37,02,01,2013,Cinema - Blu-ray,"Yaroslavl shopping center ""Altair"""
1,3,59,944,150.0,1.0,2012 (BD),37,26,04,2013,Cinema - Blu-ray,"Yaroslavl shopping center ""Altair"""
2,5,59,944,199.5,1.0,2012 (BD),37,26,06,2013,Cinema - Blu-ray,"Yaroslavl shopping center ""Altair"""
3,6,59,944,199.5,1.0,2012 (BD),37,20,07,2013,Cinema - Blu-ray,"Yaroslavl shopping center ""Altair"""
4,8,59,944,299.0,2.0,2012 (BD),37,14,09,2013,Cinema - Blu-ray,"Yaroslavl shopping center ""Altair"""
